# Data Preparation
Uses data from the Lahman database http://www.seanlahman.com/baseball-archive/statistics/ and https://www.baseball-reference.com/data/ through the 2020 season. <br>
Raw data is available in data_lahman and data_bbref folders, respectively. <br>
Raw data has been imported into a SQL database and is stored at a various levels of detail depending on the table and player. <br>
PlayerSeasonStats.sql returns selected fields and structured by player or player-season, depending on the source table. <br>
Data will be reviewed for missing values, the missing values imputed, summarized by player career and additional features will be calculated. <br>
Data is imputed prior to summarizing by career and EDA because some players spanned seasons in which certain statistics were and were not compiled. As imputation needed to happen at the season level, that was done before rolling up into a career record.<br>
Definitions of statistics https://www.baseball-reference.com/bullpen/Baseball_statistics<br>

In [1]:
# Import the libraries
import os  
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Function Definition

In [2]:
def nan_by_feature(df):
    total = df.isnull().sum()
    percent = (df.isnull().sum()/df.isnull().count()*100)
    dtypes = df.dtypes
    missing_training_data = pd.concat([total, percent, dtypes], axis=1, keys=['Total', 'Percent', 'Type'])
    missing_training_data = missing_training_data.sort_values(by=['Percent'], ascending=False)
    return missing_training_data.head(30)



def impute_nan(df, imp_cols, all_cols):
    imp = IterativeImputer(max_iter=10, random_state=0, min_value = 0,
                       sample_posterior = True, missing_values = np.nan)

    df_imp = pd.DataFrame(np.round(imp.fit_transform(df[imp_cols])))
    df_source_imp = df.join(df_imp)
    df_source_imp = df_source_imp.drop(imp_cols, axis = 1) 
    df_source_imp.columns = all_cols

    return df_source_imp

## Data Import

In [3]:
# Import data from csv files
df_people = pd.read_csv('data_sql_export/pPeople.csv')
df_HOFInductees = pd.read_csv('data_sql_export/pHOFInductees.csv')
df_HOFDetails = pd.read_csv('data_sql_export/pHOFDetails.csv')
df_batting = pd.read_csv('data_sql_export/pBatting.csv')
df_fielding = pd.read_csv('data_sql_export/pFielding.csv')
df_pitching = pd.read_csv('data_sql_export/pPitching.csv')
df_WARbatting = pd.read_csv('data_sql_export/pWARBatting.csv')
df_WARpitching = pd.read_csv('data_sql_export/pWARPitching.csv')
df_frisch = pd.read_csv('data_sql_export/pFrisch.csv')
df_awards = pd.read_csv('data_sql_export/pAwards.csv')
df_PED = pd.read_csv('data_sql_export/pPED.csv')

### People
The People table includes information about players, managers, umpires and executives. Data includes personal information such as height, weight and dates and places of birth/death,. Player information also includes batting/throwing handedness and the dates of first and last game.<br>

SQL procesing:<br>
 - Excludes NULL values in debut to exclude non-players, which also eliminates ~10 players that had very brief careers in the 19th century for which no debut date is known <br>
 - Excludes players with less than 10 years of playing experience as they don't meet the HOF requirement for consideration <br>

Fields included here: <br>
 - ID <br>
 - Name <br>
 - Batting hand<br> 
 - Throwing hand <br>
 - Debut year <br>
 - Final year <br>
 - Number of seasons in which the player appeared<br>.

In [4]:
#Preview the data
df_people.head(10)

,playerID,Name,bats,throws,debut,final,NumYears
0,aaronha01,Hank Aaron,R,R,1954,1976,23
1,aasedo01,Don Aase,R,R,1977,1990,13
2,abbotgl01,Glenn Abbott,R,R,1973,1984,11
3,abbotji01,Jim Abbott,L,L,1989,1999,10
4,abbotpa01,Paul Abbott,R,R,1990,2004,11
5,abernte02,Ted Abernathy,R,R,1955,1972,14
6,abreubo01,Bobby Abreu,L,R,1996,2014,18
7,ackerji01,Jim Acker,R,R,1983,1992,10
8,adairje01,Jerry Adair,R,R,1958,1970,13
9,adamsba01,Babe Adams,L,R,1906,1926,19


In [5]:
# Count missing values by feature
nan_by_feature(df_people)

,Total,Percent,Type
throws,1,0.028506,object
playerID,0,0.000000,object
Name,0,0.000000,object
bats,0,0.000000,object
debut,0,0.000000,int64
final,0,0.000000,int64
NumYears,0,0.000000,int64


In [6]:
# Describe the dataset
df_people.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
debut,3508.0,1962.483466,36.477272,1871.0,1937.0,1972.0,1992.0,2011.0
final,3508.0,1975.536203,35.982073,1882.0,1950.0,1985.0,2006.0,2020.0
NumYears,3508.0,13.319840,3.038163,10.0,11.0,13.0,15.0,27.0


#### Find duplicate names
The playerID value is unique to a particular player and exists in all source data. However, there are instances where more than one player has the same name, and the player name is used as a label when viewing results of modeling. This dataframe is a reference to instances where more than one eligible player has the same name in the source data.

Many of these are father and son combinations.

In [7]:
v = df_people.Name.value_counts()
df_people[df_people.Name.isin(v.index[v.gt(1)])]

,playerID,Name,bats,throws,debut,final,NumYears
41,alomasa01,Sandy Alomar,B,R,1964,1978,15
42,alomasa02,Sandy Alomar,R,R,1988,2007,20
75,armasto01,Tony Armas,R,R,1976,1989,14
76,armasto02,Tony Armas,R,R,1999,2008,10
404,burnsge01,George Burns,R,R,1911,1925,15
405,burnsge02,George Burns,R,R,1914,1929,16
474,carmofa01,Roberto Hernandez,R,R,2006,2016,11
591,colemjo04,Joe Coleman,R,R,1942,1955,10
592,colemjo05,Joe Coleman,R,R,1965,1979,15
666,cruzjo01,Jose Cruz,L,L,1970,1988,19


### HOF Inductees
The HallOfFame table includes details about all players and other personnel that have received votes for the Hall of Fame. Data includes Year, voting body, number of ballots cast/needed for induction/received, whether the candidate was inducted, category (Player, Manager, Pioneer/Executive or Umpire), and notes. <br>
SQL procesing filters to only include players who have been inducted.<br>
Fields included here: ID, Year, Voting body, and Inducted flag.


SQL procesing:<br>
 - Includes only inductees elected as players. <br>

Fields included here: <br>
 - ID <br>
 - Year <br>
 - Voting body<br> 
 - Inducted<br>.

In [8]:
#Preview the data
df_HOFInductees.head(10)

,playerID,indYear,votedBy,inducted
0,cobbty01,1936,BBWAA,1
1,ruthba01,1936,BBWAA,1
2,wagneho01,1936,BBWAA,1
3,mathech01,1936,BBWAA,1
4,johnswa01,1936,BBWAA,1
5,lajoina01,1937,BBWAA,1
6,speaktr01,1937,BBWAA,1
7,youngcy01,1937,BBWAA,1
8,alexape01,1938,BBWAA,1
9,sislege01,1939,BBWAA,1


In [9]:
# Rename yearID to avoid confusion with single season data
df_HOFInductees = df_HOFInductees.rename(columns={'yearID':'indYear'})

In [10]:
# Count missing values by feature
nan_by_feature(df_HOFInductees)

,Total,Percent,Type
playerID,0,0.0,object
indYear,0,0.0,int64
votedBy,0,0.0,object
inducted,0,0.0,int64


In [11]:
# Describe the dataset
df_HOFInductees.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
indYear,265.0,1978.916981,24.412659,1936.0,1959.0,1977.0,2000.0,2020.0
inducted,265.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


### Batting
The Batting table includes details about the batting performance of players in a given year for a given team. Players that appeared with more than one team in the same season will have a record for their perfomance with each team. <br>
SQL procesing aggregates data to player-season. <br>
Fields included are all counting statistics. No calculated fields are included in the raw data.

In [12]:
#Preview the data
df_batting.head(10)

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aardsda01,2004,11,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,aardsda01,2006,45,2,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0
2,aardsda01,2007,25,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,aardsda01,2008,47,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
4,aardsda01,2009,73,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,aardsda01,2010,53,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
6,aardsda01,2012,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
7,aardsda01,2013,43,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,aardsda01,2015,33,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
9,aaronha01,1954,122,468,58,131,27,6,13,69.0,2.0,2.0,28,39.0,NaN,3.0,6.0,4.0,13.0


In [13]:
# Rename doubles and triples features to avoid confusion with defensive games at 2B and 3B
df_batting = df_batting.rename(columns={"2B": "Doubles", "3B": "Triples"})

In [14]:
# Count missing values by feature
nan_by_feature(df_batting)

,Total,Percent,Type
IBB,34036,33.802761,float64
SF,33528,33.298242,float64
GIDP,23631,23.469063,float64
CS,21662,21.513556,float64
SH,5481,5.443440,float64
HBP,2516,2.498759,float64
SB,2104,2.089582,float64
SO,1814,1.801569,float64
RBI,588,0.583971,float64
yearID,0,0.000000,int64


In [15]:
# Impute all missing values and merge into original dataframe
df_batting_imp = impute_nan(df_batting, ['G', 'AB', 'R', 'H', 'Doubles', 'Triples', 'HR', 'RBI', 'SB', 
                                         'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP'], 
                                        df_batting.columns)

In [16]:
# Verify all missing values have been replaced with imputed values
nan_by_feature(df_batting_imp)

,Total,Percent,Type
playerID,0,0.0,object
SB,0,0.0,float64
SF,0,0.0,float64
SH,0,0.0,float64
HBP,0,0.0,float64
IBB,0,0.0,float64
SO,0,0.0,float64
BB,0,0.0,float64
CS,0,0.0,float64
RBI,0,0.0,float64


In [17]:
# Describe the dataset
df_batting_imp.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
yearID,100690.0,1967.121522,39.639992,1871.0,1936.0,1976.0,2001.0,2020.0
G,100690.0,54.821800,47.959701,1.0,15.0,37.0,89.0,165.0
AB,100690.0,150.441196,189.842619,0.0,4.0,53.0,259.0,716.0
R,100690.0,19.970216,29.020816,0.0,0.0,4.0,31.0,198.0
H,100690.0,39.315523,53.980634,0.0,0.0,10.0,65.0,262.0
Doubles,100690.0,6.700884,9.973806,0.0,0.0,1.0,10.0,67.0
Triples,100690.0,1.347383,2.695410,0.0,0.0,0.0,1.0,36.0
HR,100690.0,3.079402,6.631550,0.0,0.0,0.0,3.0,73.0
RBI,100690.0,18.104777,27.095228,0.0,0.0,4.0,28.0,191.0
SB,100690.0,3.286553,8.000328,0.0,0.0,0.0,3.0,138.0


### Fielding
The Fielding table includes details about the fielding performance of players in a given year at a given postion for a given team. <br>
SQL procesing aggregates data to player-season, aggregating the number of games played at each position and determining which position at which the player had the most games played.

In [18]:
#Preview the data
df_fielding.head(10)

,playerID,yearID,P,C,1B,2B,3B,SS,OF,LF,CF,RF,PrimPos
0,abercda01,1871,0,0,0,0,0,1,0,0,0,0,SS
1,addybo01,1871,0,0,0,22,0,3,0,0,0,0,2B
2,allisar01,1871,0,0,0,2,0,0,29,0,29,0,OF
3,allisdo01,1871,0,27,0,0,0,0,0,0,0,0,C
4,ansonca01,1871,0,5,1,2,20,0,1,1,0,0,3B
5,armstbo01,1871,0,0,0,0,0,0,12,0,11,1,OF
6,barkeal01,1871,0,0,0,0,0,0,1,1,0,0,OF
7,barnero01,1871,0,0,0,16,0,15,0,0,0,0,2B
8,barrebi01,1871,0,1,0,0,1,0,0,0,0,0,C
9,barrofr01,1871,0,0,0,1,0,0,17,13,0,4,OF


In [19]:
# Count missing values by feature
nan_by_feature(df_fielding)

,Total,Percent,Type
playerID,0,0.0,object
yearID,0,0.0,int64
P,0,0.0,int64
C,0,0.0,int64
1B,0,0.0,int64
2B,0,0.0,int64
3B,0,0.0,int64
SS,0,0.0,int64
OF,0,0.0,int64
LF,0,0.0,int64


In [20]:
# Describe the dataset
df_fielding.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
yearID,99501.0,1967.175114,39.725111,1871.0,1936.0,1976.0,2001.0,2020.0
P,99501.0,11.416589,17.983032,0.0,0.0,0.0,21.0,106.0
C,99501.0,5.076683,19.702164,0.0,0.0,0.0,0.0,160.0
1B,99501.0,4.928594,21.566947,0.0,0.0,0.0,0.0,162.0
2B,99501.0,4.913297,21.273163,0.0,0.0,0.0,0.0,163.0
3B,99501.0,4.924222,21.167266,0.0,0.0,0.0,0.0,164.0
SS,99501.0,4.888363,21.982538,0.0,0.0,0.0,0.0,165.0
OF,99501.0,14.817972,36.210463,0.0,0.0,0.0,2.0,164.0
LF,99501.0,5.202320,19.790171,0.0,0.0,0.0,0.0,163.0
CF,99501.0,4.911740,21.025356,0.0,0.0,0.0,0.0,162.0


### Pitching
The Pitching table includes details about the Pitching performance of players in a given year for a given team. Players that appeared with more than one team in the same season will have a record for their perfomance with each team. <br>
SQL procesing aggregates data to player-season. <br>
Fields included are all counting statistics. No calculated fields are included in the raw data.

In [21]:
#Preview the data
df_pitching.head(10)

,playerID,yearID,W,L,G,GS,CG,SHO,SV,IPouts,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,gomezle01,1937,21,11,34,34,25,6,0,835,...,NaN,2,1.0,0,1148.0,0,88,NaN,NaN,NaN
1,dubiemo01,1949,6,9,32,20,3,1,4,443,...,NaN,0,1.0,1,628.0,6,75,NaN,NaN,NaN
2,colemjo05,1973,23,15,40,40,13,2,0,865,...,2.0,6,10.0,0,1219.0,0,125,16.0,4.0,14.0
3,birddo01,1983,1,4,22,6,0,0,1,203,...,4.0,1,2.0,0,303.0,7,52,0.0,4.0,9.0
4,rodrifr03,2012,2,7,78,0,0,0,3,216,...,1.0,6,0.0,0,305.0,13,37,1.0,3.0,7.0
5,shantbo01,1953,5,9,16,16,6,0,0,317,...,NaN,3,0.0,0,437.0,0,52,NaN,NaN,NaN
6,ruthvdi01,1974,9,13,35,35,6,0,0,638,...,7.0,10,3.0,3,922.0,0,106,7.0,9.0,15.0
7,cornere01,2000,4,10,22,21,0,0,0,375,...,4.0,8,4.0,5,547.0,0,74,9.0,6.0,12.0
8,dewitma01,2001,0,2,16,0,0,0,0,57,...,5.0,2,1.0,0,87.0,9,8,1.0,0.0,2.0
9,millesh01,2014,10,9,32,31,1,1,0,549,...,4.0,4,2.0,0,764.0,0,78,7.0,4.0,14.0


In [22]:
# Count missing values by feature
nan_by_feature(df_pitching)

,Total,Percent,Type
GIDP,18874,42.201999,float64
SF,17829,39.865394,float64
SH,17829,39.865394,float64
IBB,13611,30.434005,float64
HBP,671,1.500347,float64
BFP,3,0.006708,float64
SO,0,0.000000,int64
R,0,0.000000,int64
GF,0,0.000000,int64
BK,0,0.000000,int64


In [23]:
# Impute all missing values and merge into original dataframe
df_pitching_imp = impute_nan(df_pitching, 
                             ['W', 'L', 'G', 'GS', 'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 
                              'IP', 'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'], 
                             df_pitching.columns)

In [24]:
# Verify all missing values have been replaced with imputed values
nan_by_feature(df_pitching_imp)

,Total,Percent,Type
playerID,0,0.0,object
SO,0,0.0,float64
SF,0,0.0,float64
SH,0,0.0,float64
R,0,0.0,float64
GF,0,0.0,float64
BFP,0,0.0,float64
BK,0,0.0,float64
HBP,0,0.0,float64
WP,0,0.0,float64


In [25]:
# Describe the dataset
df_pitching_imp.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
yearID,44723.0,1971.954274,38.421138,1871.0,1944.0,1982.0,2004.0,2020.0
W,44723.0,4.924066,5.864285,0.0,0.0,3.0,8.0,60.0
L,44723.0,4.924066,5.068786,0.0,1.0,3.0,8.0,48.0
G,44723.0,25.399928,19.086942,1.0,8.0,25.0,36.0,106.0
GS,44723.0,9.903361,12.561746,0.0,0.0,3.0,19.0,75.0
CG,44723.0,3.172216,7.177261,0.0,0.0,0.0,2.0,75.0
SHO,44723.0,0.452899,1.117184,0.0,0.0,0.0,0.0,16.0
SV,44723.0,1.603314,5.189154,0.0,0.0,0.0,1.0,62.0
IPouts,44723.0,265.023992,259.343431,0.0,55.0,183.0,419.0,2040.0
H,44723.0,88.499385,85.471247,0.0,20.0,59.0,142.0,772.0


### Batting WAR
Uses data from the war_daily_bat.txt extract from baseballreference.com. 

https://www.baseball-reference.com/data/war_daily_bat.txt

This file includes details about all the components which go into calculating a position player's Wins Above Replacement (WAR). WAR is a measure of a player's value compared to a player that would be freely available for acquisition. For more details: https://www.baseball-reference.com/bullpen/Wins_Above_Replacement


Fields included: <br>
 - ID <br>
 - Year <br>
 - WAR<br>.

In [26]:
#Preview the data
df_WARbatting.head(10)

,playerID,year_ID,WAR
0,aaronha01,1954,1.42
1,aaronha01,1955,6.24
2,aaronha01,1956,7.18
3,aaronha01,1957,7.96
4,aaronha01,1958,7.32
5,aaronha01,1959,8.64
6,aaronha01,1960,8.00
7,aaronha01,1961,9.45
8,aaronha01,1962,8.54
9,aaronha01,1963,9.12


In [27]:
# Count missing values by feature
nan_by_feature(df_WARbatting)

,Total,Percent,Type
WAR,3,0.004869,float64
playerID,1,0.001623,object
year_ID,0,0.000000,int64


In [40]:
df_WARbatting = df_WARbatting.dropna()

In [41]:
# Count missing values by feature
nan_by_feature(df_WARbatting)

,Total,Percent,Type
playerID,0,0.0,object
year_ID,0,0.0,int64
WAR,0,0.0,float64


In [42]:
# Describe the dataset
df_WARbatting.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
year_ID,61604.0,1962.627946,40.932087,1871.00,1929.00,1971.00,1998.00,2020.00
WAR,61604.0,0.855737,1.726744,-4.02,-0.16,0.16,1.41,14.07


### Pitching WAR
Uses data from the war_daily_pitch.txt extract from baseballreference.com. 

https://www.baseball-reference.com/data/war_daily_pitch.txt

This file includes details about all the components which go into calculating a pitcher's Wins Above Replacement (WAR). WAR is a measure of a pitcher's value compared to a pitcher that would be freely available for acquisition. For more details: https://www.baseball-reference.com/bullpen/Wins_Above_Replacement <br>

SQL Processing excludes pitcher seasons with zero IPOuts. WAR cannot be calculated unless a pitcher retires at least one batter.

Fields included: <br>
 - ID <br>
 - Year <br>
 - WAR<br>.

In [43]:
#Preview the data
df_WARpitching.head(10)

,playerID,year_ID,WAR
0,aardsda01,2004,-0.15
1,aardsda01,2006,0.61
2,aardsda01,2007,-0.40
3,aardsda01,2008,-0.31
4,aardsda01,2009,1.69
5,aardsda01,2010,0.68
6,aardsda01,2012,-0.02
7,aardsda01,2013,-0.13
8,aardsda01,2015,-0.12
9,aasedo01,1977,2.39


In [44]:
# Count missing values by feature
nan_by_feature(df_WARpitching)

,Total,Percent,Type
playerID,0,0.0,object
year_ID,0,0.0,int64
WAR,0,0.0,float64


In [45]:
# Describe the dataset
df_WARpitching.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
year_ID,48282.0,1972.243362,38.39054,1871.00,1945.00,1983.00,2005.00,2020.00
WAR,48282.0,0.758624,1.72754,-5.46,-0.22,0.17,1.25,20.48


### Awards
The AwardsPlayers table includes information about the winners of various awards each year. Awards may have a single winner in MLB, one winner in each league, or one winner at each position in each league.<br>
SQL procesing transforms the data to player-season format, for any player that won one or more of the following awards: <br>

 - Rookie of the Year
 - Most Valuable Player
 - Cy Young Award
 - Triple Crown
 - Gold Glove for each position
 
 

In [46]:
#Preview the data
df_awards.head(10)

,playerID,yearID,ROY,MVP,CYA,TC,GG_P,GG_C,GG_1B,GG_2B,GG_3B,GG_SS,GG_LF,GG_CF,GG_RF,GG_OF
0,aaronha01,1957,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,aaronha01,1958,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,aaronha01,1959,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,aaronha01,1960,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,abreubo01,2005,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,abreujo02,2014,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,ageeto01,1966,1,0,0,0,0,0,0,0,0,0,0,0,0,1
7,ageeto01,1970,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,allendi01,1964,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,allendi01,1972,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
# Count missing values by feature
nan_by_feature(df_awards)

,Total,Percent,Type
playerID,0,0.0,object
yearID,0,0.0,int64
ROY,0,0.0,int64
MVP,0,0.0,int64
CYA,0,0.0,int64
TC,0,0.0,int64
GG_P,0,0.0,int64
GG_C,0,0.0,int64
GG_1B,0,0.0,int64
GG_2B,0,0.0,int64


In [48]:
# Describe the dataset
df_awards.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
yearID,1480.0,1984.141892,21.532939,1878.0,1969.0,1985.0,2002.0,2017.0
ROY,1480.0,0.095946,0.294617,0.0,0.0,0.0,0.0,1.0
MVP,1480.0,0.132432,0.339075,0.0,0.0,0.0,0.0,1.0
CYA,1480.0,0.077027,0.266724,0.0,0.0,0.0,0.0,1.0
TC,1480.0,0.011486,0.106594,0.0,0.0,0.0,0.0,1.0
GG_P,1480.0,0.081757,0.274086,0.0,0.0,0.0,0.0,1.0
GG_C,1480.0,0.081757,0.274086,0.0,0.0,0.0,0.0,1.0
GG_1B,1480.0,0.081757,0.274086,0.0,0.0,0.0,0.0,1.0
GG_2B,1480.0,0.081757,0.274086,0.0,0.0,0.0,0.0,1.0
GG_3B,1480.0,0.081757,0.274086,0.0,0.0,0.0,0.0,1.0


## Career Summaries
The following dataframes will be aggregated from player-season detail to the player level to summarize each player's career: <br>
 - Batting
 - Fielding
 - Pitching
 - Awards
 - WAR
 
Dataframes will be merged to result in one dataframe for position players and one for pitchers. Each will include only players with at least 10 years of playing time and include calculations of various rate statistics.

### Position Players

In [49]:
# Aggregate all columns by playerID and drop yearID
df_batting_career = df_batting_imp.groupby(by='playerID').sum()
df_batting_career = df_batting_career.drop('yearID', axis = 1) 

## Calculated Batting Features <br>
For each player's career, calculate the following features:<br>

- Plate Appearances (PA)
- Batting Average (Avg)
- On Base Percentage (OBP)
- Slugging Percentage (Slg)
- On Base + Slugging (OPS)
- Total Bases (TB)

In [50]:
df_batting_career['PA'] = df_batting_career.AB + df_batting_career.BB + df_batting_career.HBP + df_batting_career.SH + df_batting_career.SF
df_batting_career['TB'] = df_batting_career.H + df_batting_career.Doubles + (df_batting_career.Triples * 2) + (df_batting_career.HR * 3)
df_batting_career['Avg'] = df_batting_career.H / df_batting_career.AB
df_batting_career['OBP'] = (df_batting_career.H + df_batting_career.BB + df_batting_career.HBP) / (df_batting_career.AB + df_batting_career.BB + df_batting_career.HBP + df_batting_career.SF)
df_batting_career['Slg'] = df_batting_career.TB / df_batting_career.AB
df_batting_career['OPS'] = df_batting_career.OBP + df_batting_career.Slg
df_batting_career['SBA'] = df_batting_career.SB + df_batting_career.CS
df_batting_career['SB_Pct'] = df_batting_career.SB / (df_batting_career.SB + df_batting_career.CS)

In [51]:
df_batting_career.head(10)

,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,...,SF,GIDP,PA,TB,Avg,OBP,Slg,OPS,SBA,SB_Pct
playerID,,,,,,,,,,,,,,,,,,,,,
aardsda01,331.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,NaN
aaronha01,3298.0,12364.0,2174.0,3771.0,624.0,98.0,755.0,2297.0,240.0,73.0,...,121.0,328.0,13940.0,6856.0,0.304998,0.373949,0.554513,0.928462,313.0,0.766773
aaronto01,437.0,944.0,102.0,216.0,42.0,6.0,13.0,94.0,9.0,8.0,...,6.0,36.0,1045.0,309.0,0.228814,0.291506,0.327331,0.618836,17.0,0.529412
aasedo01,448.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,NaN
abadan01,15.0,21.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,25.0,2.0,0.095238,0.240000,0.095238,0.335238,1.0,0.000000
abadfe01,384.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,9.0,1.0,0.111111,0.111111,0.111111,0.222222,0.0,NaN
abadijo01,12.0,49.0,4.0,11.0,0.0,0.0,0.0,5.0,1.0,0.0,...,2.0,0.0,55.0,11.0,0.224490,0.259259,0.224490,0.483749,1.0,1.000000
abbated01,855.0,3044.0,355.0,772.0,99.0,43.0,11.0,324.0,142.0,59.0,...,28.0,64.0,3487.0,990.0,0.253614,0.322334,0.325230,0.647563,201.0,0.706468
abbeybe01,79.0,225.0,21.0,38.0,3.0,3.0,0.0,17.0,3.0,11.0,...,6.0,7.0,261.0,47.0,0.168889,0.234127,0.208889,0.443016,14.0,0.214286


In [52]:
# Aggregate all columns by playerID, drop yearID and add primary position
df_fielding_career = df_fielding.groupby(by='playerID').sum()
df_fielding_career = df_fielding_career.drop('yearID', axis = 1) 
df_fielding_career['PrimPos'] = df_fielding_career.idxmax(axis=1)

In [53]:
df_fielding_career.head(10)

,P,C,1B,2B,3B,SS,OF,LF,CF,RF,PrimPos
playerID,,,,,,,,,,,
aardsda01,331,0,0,0,0,0,0,0,0,0,P
aaronha01,0,0,210,43,7,0,2760,315,308,2174,OF
aaronto01,0,0,232,7,10,0,138,135,1,2,1B
aasedo01,448,0,0,0,0,0,0,0,0,0,P
abadan01,0,0,8,0,0,0,1,0,0,1,1B
abadfe01,384,0,0,0,0,0,0,0,0,0,P
abadijo01,0,0,12,0,0,0,0,0,0,0,1B
abbated01,0,0,0,419,20,388,3,0,2,1,2B
abbeybe01,79,0,0,0,0,0,0,0,0,0,P


In [54]:
# Aggregate all columns by playerID and drop yearID
df_awards_career = df_awards.groupby(by='playerID').sum()
df_awards_career = df_awards_career.drop('yearID', axis = 1) 

Gold Gloves for outfielders have been awarded inconsistently. In some seasons, awards would be given to one left fielder, one center fielder and one right fielder. In other seasons, awards are given to three outfielders regardless of their position. 

For example, Al Kaline won 10 Gold Gloves: One as a CF, two as a RF and seven as an OF.

In [55]:
# Aggregate all OF Gold Gloves
df_awards_career['GG_All_OF'] = df_awards_career.GG_LF + df_awards_career.GG_CF + df_awards_career.GG_RF + \
                                df_awards_career.GG_OF

In [56]:
# Aggregate total Gold Gloves at all positions
df_awards_career['GG_Total'] = df_awards_career.GG_C + df_awards_career.GG_1B + df_awards_career.GG_2B + \
                                df_awards_career.GG_3B + df_awards_career.GG_SS + df_awards_career.GG_All_OF

In [57]:
df_awards_career.head(10)

,ROY,MVP,CYA,TC,GG_P,GG_C,GG_1B,GG_2B,GG_3B,GG_SS,GG_LF,GG_CF,GG_RF,GG_OF,GG_All_OF,GG_Total
playerID,,,,,,,,,,,,,,,,
aaronha01,0,1,0,0,0,0,0,0,0,0,0,0,3,0,3,3
abreubo01,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
abreujo02,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ageeto01,1,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2
allendi01,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
alleyge01,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2
allisbo01,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
alomaro01,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,10
alomasa02,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [58]:
# Aggregate WAR by playerID and drop yearID
df_WARbatting_career = df_WARbatting.groupby(by='playerID').sum()
df_WARbatting_career = df_WARbatting_career.drop('year_ID', axis = 1) 

In [59]:
df_WARbatting_career.head(10)

,WAR
playerID,
aaronha01,143.10
aaronto01,-2.81
abadan01,-0.37
abadijo01,-0.06
abbated01,8.61
abbeych01,1.79
abbotfr01,0.54
abbotje01,-1.43
abbotku01,0.54


In [60]:
# Merge the People dataframe with batting, fielding, and awards career tables, 
# then add HOF Inductee information and drop players that were primarily pitchers
df_pos_play = df_people[['playerID', 'Name', 'NumYears', 'debut', 'final']]
df_pos_play = pd.merge(df_pos_play, df_batting_career, how='left', on='playerID')
df_pos_play = pd.merge(df_pos_play, df_fielding_career, how='left', on='playerID')
df_pos_play = pd.merge(df_pos_play, df_awards_career, how='left', on='playerID')
df_pos_play = pd.merge(df_pos_play, df_WARbatting_career, how='left', on='playerID')
df_pos_play = pd.merge(df_pos_play, df_HOFInductees, how='left', on='playerID')
df_pos_play.drop(df_pos_play[df_pos_play['PrimPos'] == 'P'].index, inplace = True)
df_pos_play = df_pos_play.set_index('playerID')

In [61]:
# Drop the features specific to pitchers (Cy Young Award and pitcher Gold Gloves)
pit_cols = ['CYA', 'GG_P']
df_pos_play = df_pos_play.drop(pit_cols, axis = 1) 

In [62]:
df_pos_play.head(10)

,Name,NumYears,debut,final,G,AB,R,H,Doubles,Triples,...,GG_LF,GG_CF,GG_RF,GG_OF,GG_All_OF,GG_Total,WAR,indYear,votedBy,inducted
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,Hank Aaron,23,1954,1976,3298.0,12364.0,2174.0,3771.0,624.0,98.0,...,0.0,0.0,3.0,0.0,3.0,3.0,143.10,1982.0,BBWAA,1.0
abreubo01,Bobby Abreu,18,1996,2014,2425.0,8480.0,1453.0,2470.0,574.0,59.0,...,0.0,0.0,0.0,1.0,1.0,1.0,60.18,NaN,NaN,NaN
adairje01,Jerry Adair,13,1958,1970,1165.0,4019.0,378.0,1022.0,163.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,8.38,NaN,NaN,NaN
adamsbo03,Bobby Adams,14,1946,1959,1281.0,4019.0,591.0,1082.0,188.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,15.65,NaN,NaN,NaN
adamssp01,Sparky Adams,13,1922,1934,1424.0,5557.0,844.0,1588.0,249.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,16.63,NaN,NaN,NaN
adcocjo01,Joe Adcock,17,1950,1966,1959.0,6606.0,823.0,1832.0,295.0,35.0,...,NaN,NaN,NaN,NaN,NaN,NaN,33.59,NaN,NaN,NaN
ageeto01,Tommie Agee,12,1962,1973,1129.0,3912.0,558.0,999.0,170.0,27.0,...,0.0,0.0,0.0,2.0,2.0,2.0,25.36,NaN,NaN,NaN
aguaylu01,Luis Aguayo,10,1980,1989,568.0,1104.0,142.0,260.0,43.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.71,NaN,NaN,NaN
ainsmed01,Eddie Ainsmith,15,1910,1924,1078.0,3048.0,299.0,707.0,108.0,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6.33,NaN,NaN,NaN


In [63]:
# Count missing values by feature
nan_by_feature(df_pos_play)

,Total,Percent,Type
inducted,1998,92.500000,float64
votedBy,1998,92.500000,object
indYear,1998,92.500000,float64
GG_SS,1764,81.666667,float64
ROY,1764,81.666667,float64
MVP,1764,81.666667,float64
TC,1764,81.666667,float64
GG_C,1764,81.666667,float64
GG_2B,1764,81.666667,float64
GG_3B,1764,81.666667,float64


All NaN values are for awards or HOF data, for players that don't have data in those table. These can be replaced with zeros.

In [64]:
df_pos_play = df_pos_play.fillna(0)
df_pos_play.head(10)

,Name,NumYears,debut,final,G,AB,R,H,Doubles,Triples,...,GG_LF,GG_CF,GG_RF,GG_OF,GG_All_OF,GG_Total,WAR,indYear,votedBy,inducted
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,Hank Aaron,23,1954,1976,3298.0,12364.0,2174.0,3771.0,624.0,98.0,...,0.0,0.0,3.0,0.0,3.0,3.0,143.10,1982.0,BBWAA,1.0
abreubo01,Bobby Abreu,18,1996,2014,2425.0,8480.0,1453.0,2470.0,574.0,59.0,...,0.0,0.0,0.0,1.0,1.0,1.0,60.18,0.0,0,0.0
adairje01,Jerry Adair,13,1958,1970,1165.0,4019.0,378.0,1022.0,163.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.38,0.0,0,0.0
adamsbo03,Bobby Adams,14,1946,1959,1281.0,4019.0,591.0,1082.0,188.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.65,0.0,0,0.0
adamssp01,Sparky Adams,13,1922,1934,1424.0,5557.0,844.0,1588.0,249.0,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.63,0.0,0,0.0
adcocjo01,Joe Adcock,17,1950,1966,1959.0,6606.0,823.0,1832.0,295.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33.59,0.0,0,0.0
ageeto01,Tommie Agee,12,1962,1973,1129.0,3912.0,558.0,999.0,170.0,27.0,...,0.0,0.0,0.0,2.0,2.0,2.0,25.36,0.0,0,0.0
aguaylu01,Luis Aguayo,10,1980,1989,568.0,1104.0,142.0,260.0,43.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.71,0.0,0,0.0
ainsmed01,Eddie Ainsmith,15,1910,1924,1078.0,3048.0,299.0,707.0,108.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.33,0.0,0,0.0


In [65]:
# Describe the dataset
df_pos_play.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NumYears,2160.0,13.439815,3.059710,10.000000,11.000000,13.000000,15.000000,27.000000
debut,2160.0,1958.803704,37.913573,1871.000000,1931.000000,1968.000000,1990.000000,2011.000000
final,2160.0,1971.932407,37.372405,1882.000000,1945.000000,1982.000000,2004.000000,2020.000000
G,2160.0,1334.557407,528.868601,140.000000,954.750000,1280.000000,1652.000000,3562.000000
AB,2160.0,4524.001852,2145.018423,252.000000,2883.500000,4287.000000,5813.250000,14053.000000
R,2160.0,636.240741,376.499839,20.000000,349.000000,578.000000,844.000000,2295.000000
H,2160.0,1238.427315,653.862333,48.000000,740.500000,1154.500000,1597.000000,4256.000000
Doubles,2160.0,215.668981,123.221612,6.000000,121.750000,195.000000,283.000000,792.000000
Triples,2160.0,41.799537,37.236872,0.000000,16.000000,32.000000,56.000000,309.000000
HR,2160.0,106.713426,111.206516,0.000000,28.000000,69.000000,144.000000,762.000000


### Pitchers

In [66]:
# Aggregate all columns by playerID, drop yearID and add primary position
df_pitching_career = df_pitching_imp.groupby(by='playerID').sum()
df_pitching_career = df_pitching_career.drop('yearID', axis = 1) 

In [67]:
# Drop IP and ERA, which will be recalcuated based on career totals
df_pitching_career = df_pitching_career.drop(['IP', 'ERA'], axis = 1) 

## Calculated Pitching Features <br>
For each player's career, calculate the following features:<br>

- Innings Pitched (IP)
- Earned Run Average (ERA)
- Winning Percentate (WPct)
- Walks + Hits per Inning Pitched (WHIP)
- Hits per 9 innings (H9)
- Home Runs per 9 innings(HR9)
- Walks per 9 innings(BB9)
- Strikeouts per 9 innings(K9)
- Strikeouts per walk(SOW)

In [68]:
df_pitching_career['IP'] = df_pitching_career.IPouts / 3
df_pitching_career['ERA'] = (df_pitching_career.ER / df_pitching_career.IP) * 9
df_pitching_career['WPct'] = df_pitching_career.W / (df_pitching_career.W + df_pitching_career.L)
df_pitching_career['WHIP'] = (df_pitching_career.BB + df_pitching_career.H) / df_pitching_career.IP
df_pitching_career['H9'] = (df_pitching_career.H / df_pitching_career.IP) * 9
df_pitching_career['HR9'] = (df_pitching_career.HR / df_pitching_career.IP) * 9
df_pitching_career['BB9'] = (df_pitching_career.BB / df_pitching_career.IP) * 9
df_pitching_career['K9'] = (df_pitching_career.SO / df_pitching_career.IP) * 9
df_pitching_career['SOW'] = df_pitching_career.SO / df_pitching_career.BB
# Drop IPOuts as it's now redundant with IP
df_pitching_career = df_pitching_career.drop(['IPouts'], axis = 1) 

In [69]:
df_pitching_career.head(10)

,W,L,G,GS,CG,SHO,SV,H,ER,HR,...,GIDP,IP,ERA,WPct,WHIP,H9,HR9,BB9,K9,SOW
playerID,,,,,,,,,,,,,,,,,,,,,
aardsda01,16.0,18.0,331.0,0.0,0.0,0.0,69.0,296.0,160.0,41.0,...,21.0,337.000000,4.272997,0.470588,1.421365,7.905045,1.094955,4.887240,9.080119,1.857923
aasedo01,66.0,60.0,448.0,91.0,22.0,5.0,82.0,1085.0,468.0,89.0,...,106.0,1109.333333,3.796875,0.523810,1.390024,8.802584,0.722055,3.707632,5.200421,1.402626
abadfe01,8.0,29.0,384.0,6.0,0.0,0.0,2.0,309.0,135.0,42.0,...,25.0,330.666667,3.674395,0.216216,1.285282,8.410282,1.143145,3.157258,7.620968,2.413793
abbeybe01,22.0,40.0,79.0,65.0,52.0,0.0,1.0,686.0,285.0,18.0,...,71.0,568.000000,4.515845,0.354839,1.545775,10.869718,0.285211,3.042254,2.551056,0.838542
abbeych01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,...,6.0,2.000000,4.500000,NaN,3.000000,27.000000,0.000000,0.000000,0.000000,NaN
abbotda01,0.0,2.0,3.0,1.0,1.0,0.0,1.0,19.0,9.0,0.0,...,1.0,13.000000,6.230769,0.000000,2.076923,13.153846,0.000000,5.538462,0.692308,0.125000
abbotgl01,62.0,83.0,248.0,206.0,37.0,5.0,0.0,1405.0,627.0,162.0,...,111.0,1286.000000,4.388025,0.427586,1.366252,9.832815,1.133748,2.463453,3.387247,1.375000
abbotji01,87.0,108.0,263.0,254.0,31.0,6.0,0.0,1779.0,791.0,154.0,...,200.0,1674.000000,4.252688,0.446154,1.433094,9.564516,0.827957,3.333333,4.774194,1.432258
abbotky01,4.0,17.0,57.0,22.0,0.0,0.0,0.0,207.0,107.0,26.0,...,18.0,185.333333,5.196043,0.190476,1.543165,10.052158,1.262590,3.836331,6.021583,1.569620


In [70]:
# Aggregate WAR by playerID and drop yearID
df_WARpitching_career = df_WARpitching.groupby(by='playerID').sum()
df_WARpitching_career = df_WARpitching_career.drop('year_ID', axis = 1) 

In [71]:
df_WARpitching_career.head(10)

,WAR
playerID,
aardsda01,1.85
aasedo01,15.09
abadfe01,3.27
abbeybe01,1.32
abbeych01,-0.09
abbotda01,-0.37
abbotgl01,5.26
abbotji01,19.70
abbotky01,-1.37


In [72]:
# Merge the People dataframe with pitching, fielding, and awards career tables, 
# then add HOF Inductee information and drop players that were primarily position players
df_pitchers = df_people[['playerID', 'Name', 'NumYears', 'debut', 'final']]
df_pitchers = pd.merge(df_pitchers, df_pitching_career, how='left', on='playerID')
df_pitchers = pd.merge(df_pitchers, df_fielding_career, how='left', on='playerID')
df_pitchers = pd.merge(df_pitchers, df_awards_career, how='left', on='playerID')
df_pitchers = pd.merge(df_pitchers, df_WARpitching_career, how='left', on='playerID')
df_pitchers = pd.merge(df_pitchers, df_HOFInductees, how='left', on='playerID')
df_pitchers.drop(df_pitchers[df_pitchers['PrimPos'] != 'P'].index, inplace = True)
df_pitchers = df_pitchers.set_index('playerID')

In [73]:
# Drop the features specific to position players
# Triple Crown award cannot be won by a pitcher
df_pitchers = df_pitchers.drop('TC', axis = 1) 

# Gold Gloves for positions other than pitcher
gg_cols = ['GG_C', 'GG_1B', 'GG_2B', 'GG_3B', 'GG_SS', 'GG_LF', 'GG_CF', 'GG_RF', 'GG_OF', 'GG_All_OF', 'GG_Total']
df_pitchers = df_pitchers.drop(gg_cols, axis = 1) 

#Games played at positions other than pitcher
pos_cols = ['C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'OF']
df_pitchers = df_pitchers.drop(pos_cols, axis = 1) 

# Drop the PrimPos feature, which has the same value for all rows
df_pitchers = df_pitchers.drop('PrimPos', axis = 1) 

In [74]:
df_pitchers.head(10)

,Name,NumYears,debut,final,W,L,G,GS,CG,SHO,...,SOW,P,ROY,MVP,CYA,GG_P,WAR,indYear,votedBy,inducted
playerID,,,,,,,,,,,,,,,,,,,,,
aasedo01,Don Aase,13,1977,1990,66.0,60.0,448.0,91.0,22.0,5.0,...,1.402626,448,NaN,NaN,NaN,NaN,15.09,NaN,NaN,NaN
abbotgl01,Glenn Abbott,11,1973,1984,62.0,83.0,248.0,206.0,37.0,5.0,...,1.375000,248,NaN,NaN,NaN,NaN,5.26,NaN,NaN,NaN
abbotji01,Jim Abbott,10,1989,1999,87.0,108.0,263.0,254.0,31.0,6.0,...,1.432258,263,NaN,NaN,NaN,NaN,19.70,NaN,NaN,NaN
abbotpa01,Paul Abbott,11,1990,2004,43.0,37.0,162.0,112.0,1.0,0.0,...,1.262087,162,NaN,NaN,NaN,NaN,4.48,NaN,NaN,NaN
abernte02,Ted Abernathy,14,1955,1972,63.0,69.0,681.0,34.0,7.0,2.0,...,1.292230,681,NaN,NaN,NaN,NaN,16.58,NaN,NaN,NaN
ackerji01,Jim Acker,10,1983,1992,33.0,49.0,467.0,32.0,0.0,0.0,...,1.465046,467,NaN,NaN,NaN,NaN,5.47,NaN,NaN,NaN
adamsba01,Babe Adams,19,1906,1926,194.0,140.0,482.0,355.0,206.0,44.0,...,2.409302,482,NaN,NaN,NaN,NaN,50.01,NaN,NaN,NaN
adamsmi03,Mike Adams,10,2004,2014,21.0,20.0,408.0,0.0,0.0,0.0,...,3.246032,408,NaN,NaN,NaN,NaN,10.10,NaN,NaN,NaN
adamste01,Terry Adams,11,1995,2005,51.0,62.0,574.0,41.0,0.0,0.0,...,1.818421,574,NaN,NaN,NaN,NaN,4.45,NaN,NaN,NaN


In [75]:
# Count missing values by feature
nan_by_feature(df_pitchers)

,Total,Percent,Type
inducted,1275,94.584570,float64
votedBy,1275,94.584570,object
indYear,1275,94.584570,float64
GG_P,1231,91.320475,float64
CYA,1231,91.320475,float64
MVP,1231,91.320475,float64
ROY,1231,91.320475,float64
H9,0,0.000000,float64
SF,0,0.000000,float64
GIDP,0,0.000000,float64


All NaN values are for awards or HOF data, for players that don't have data in those tablesnever won any awards. These can be replaced with zeros.

In [76]:
df_pitchers = df_pitchers.fillna(0)
df_pitchers.head(10)

,Name,NumYears,debut,final,W,L,G,GS,CG,SHO,...,SOW,P,ROY,MVP,CYA,GG_P,WAR,indYear,votedBy,inducted
playerID,,,,,,,,,,,,,,,,,,,,,
aasedo01,Don Aase,13,1977,1990,66.0,60.0,448.0,91.0,22.0,5.0,...,1.402626,448,0.0,0.0,0.0,0.0,15.09,0.0,0,0.0
abbotgl01,Glenn Abbott,11,1973,1984,62.0,83.0,248.0,206.0,37.0,5.0,...,1.375000,248,0.0,0.0,0.0,0.0,5.26,0.0,0,0.0
abbotji01,Jim Abbott,10,1989,1999,87.0,108.0,263.0,254.0,31.0,6.0,...,1.432258,263,0.0,0.0,0.0,0.0,19.70,0.0,0,0.0
abbotpa01,Paul Abbott,11,1990,2004,43.0,37.0,162.0,112.0,1.0,0.0,...,1.262087,162,0.0,0.0,0.0,0.0,4.48,0.0,0,0.0
abernte02,Ted Abernathy,14,1955,1972,63.0,69.0,681.0,34.0,7.0,2.0,...,1.292230,681,0.0,0.0,0.0,0.0,16.58,0.0,0,0.0
ackerji01,Jim Acker,10,1983,1992,33.0,49.0,467.0,32.0,0.0,0.0,...,1.465046,467,0.0,0.0,0.0,0.0,5.47,0.0,0,0.0
adamsba01,Babe Adams,19,1906,1926,194.0,140.0,482.0,355.0,206.0,44.0,...,2.409302,482,0.0,0.0,0.0,0.0,50.01,0.0,0,0.0
adamsmi03,Mike Adams,10,2004,2014,21.0,20.0,408.0,0.0,0.0,0.0,...,3.246032,408,0.0,0.0,0.0,0.0,10.10,0.0,0,0.0
adamste01,Terry Adams,11,1995,2005,51.0,62.0,574.0,41.0,0.0,0.0,...,1.818421,574,0.0,0.0,0.0,0.0,4.45,0.0,0,0.0


In [77]:
# Describe the dataset
df_pitchers.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NumYears,1348.0,13.127596,2.994434,10.000000,11.000000,12.000000,15.000000,27.000000
debut,1348.0,1968.379822,33.223720,1871.000000,1947.750000,1977.000000,1995.000000,2011.000000
final,1348.0,1981.310831,32.833009,1884.000000,1960.750000,1990.000000,2008.000000,2020.000000
W,1348.0,100.636499,66.808994,8.000000,50.000000,86.000000,132.000000,511.000000
L,1348.0,90.848665,50.027083,9.000000,51.000000,87.000000,118.000000,315.000000
G,1348.0,441.309347,172.694503,92.000000,315.000000,403.000000,541.250000,1252.000000
GS,1348.0,198.146142,157.212193,0.000000,46.000000,198.500000,300.000000,815.000000
CG,1348.0,61.123145,91.119897,0.000000,2.000000,21.000000,82.000000,749.000000
SHO,1348.0,10.247774,12.823838,0.000000,0.000000,6.000000,15.250000,110.000000
SV,1348.0,35.515579,68.303369,0.000000,3.000000,11.000000,32.000000,652.000000


### Position Players

The Hall of Fame also has an eligibility requirement that a player must have a five year waiting period before being added to the ballot. If a player is inducted in his first season of eligibility, his induction year is actually six years after his final game. <br>
Players who have played more recently will be split into their own dataset that is omitted from training. These more recent players may be used to predict induction results in upcoming years.

In [78]:
# Split the dataset into players retired at least 5 years and more recent players

# Find the most recent final season for inducted players. 
max_ind = df_HOFInductees['indYear'].max()

# Create a dataframe for recent position players
df_pos_play_recent = df_pos_play[df_pos_play['final'] > max_ind - 6] 

# Exclude these recent players from the data that will be used for modeling
df_pos_play = df_pos_play[df_pos_play['final'] <= max_ind - 6] 

#From this dataframe, also create a dataframe of known PED users
df_pos_play_ped = pd.merge(df_pos_play, df_PED, how='inner', on='playerID')
df_pos_play_ped = df_pos_play_ped.set_index('playerID')


In [79]:
# Write these datasets to csv files
df_pos_play.to_csv(r'data_career\pos_play.csv')
df_pos_play_recent.to_csv(r'data_career\pos_play_recent.csv')
df_pos_play_ped.to_csv(r'data_career\pos_play_ped.csv')

### Pitchers

In [80]:
# Split the dataset into pitcher retired at least 5 years and more recent pitcher

# Find the most recent year of induction. 
max_ind = df_HOFInductees['indYear'].max()

# Create a dataframe for recent position players
df_pitchers_recent = df_pitchers[df_pitchers['final'] > max_ind - 6] 

# Exclude these recent players from the data that will be used for modeling
df_pitchers = df_pitchers[df_pitchers['final'] <= max_ind - 6] 

#From this dataframe, also create a dataframe of known PED users
df_pitchers_ped = pd.merge(df_pitchers, df_PED, how='inner', on='playerID')
df_pitchers_ped = df_pitchers_ped.set_index('playerID')

In [81]:
# Write these datasets to csv files
df_pitchers.to_csv(r'data_career\pitchers.csv')
df_pitchers_recent.to_csv(r'data_career\pitchers_recent.csv')
df_pitchers_ped.to_csv(r'data_career\pitchers_ped.csv')